## Weight & Biases
Hyperparameter Tuning mit wandb

### Sweep Config
Konfiguration des wand sweep mit den gewünschten Hyperparameter

In [1]:
import wandb

wandb.login()

sweep_config = {
  'name' : 'GradientBoosting-Loss-GF-LAD',
  'method' : 'random',
  'parameters' : {
    'n_estimators' : {
      'values' : [256, 512, 768, 1024]
    },
    'learning_rate' :{
      'min': 0.01,
      'max': 0.15
    },
    'subsample' :{
      'min': 0.5,
      'max': 0.85
    },
    'max_depth' :{
      'values' : [4, 5, 6, 7, 8, 10, 12]
    },
    'min_samples_split' :{
      'values' : [4, 5, 6, 8, 10]
    }
  }
}

sweep_id = wandb.sweep(sweep_config, project='Metriken Bauwesen Sweeps', entity='devcore')

wandb: Currently logged in as: devcore (use `wandb login --relogin` to force relogin)


Create sweep with ID: u3louhfl
Sweep URL: https://wandb.ai/devcore/Metriken%20Bauwesen%20Sweeps/sweeps/u3louhfl


### Run Sweep
Sweep auf Agent laufen lassen für Hyperparameter-Tuning und Logging auf wandb

In [2]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import cross_validate, RepeatedKFold

import src.package.consts as c
import src.package.importer as im
import src.package.ml_helper as ml_helper
import src.package.importer_usages as imp_usg
from numpy import mean

# load dataset
df = im.get_extended_dataset('../package/datasets/train_set.csv')
df, usage_types = imp_usg.extract_usage_details(df, highest_only=True, combine_garages=True)
df = ml_helper.remove_outliers(df, factor=2.3)

X, y = ml_helper.ml_dataset_full(df, field_to_predict=c.FIELD_AREA_TOTAL_FLOOR_416, additional_features=[c.FIELD_AREA_MAIN_USAGE])

def train():
    config = {'model': 'GradientBoost', 'run': 'none'}
    with wandb.init(config=config):

        # model with hyperparameter
        config = wandb.config
        regr = GradientBoostingRegressor(random_state=0,
                                         learning_rate=config.learning_rate,
                                         subsample=config.subsample,
                                         n_estimators=config.n_estimators,
                                         max_depth=config.max_depth,
                                         min_samples_split=config.min_samples_split,
                                         loss='lad')

        # evaluate with cv and log to wandb
        cv=RepeatedKFold(n_splits=5, n_repeats=3, random_state=0)
        scores = cross_validate(regr, X, y, cv=cv, scoring=['r2', 'neg_mean_absolute_percentage_error', 'neg_mean_absolute_error', 'neg_root_mean_squared_error'])
        wandb.log({'r2': mean(scores['test_r2'])})
        wandb.log({'neg_mape': mean(scores['test_neg_mean_absolute_percentage_error'])})
        wandb.log({'neg_mae': mean(scores['test_neg_mean_absolute_error'])})
        wandb.log({'neg_rmse': mean(scores['test_neg_root_mean_squared_error'])})

wandb.agent(sweep_id, train, count=100)

Location: D:\Development\FHNW\ip5-metriken-bauwesen\export\cluster_means.joblib
Location: D:\Development\FHNW\ip5-metriken-bauwesen\export\fitted_pipeline.joblib


wandb: Agent Starting Run: 90okubuo with config:
wandb: 	learning_rate: 0.07514669200842505
wandb: 	max_depth: 6
wandb: 	min_samples_split: 10
wandb: 	n_estimators: 512
wandb: 	subsample: 0.5981791615951815
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Agent Starting Run: mpe241al with config:
wandb: 	learning_rate: 0.03551437250733531
wandb: 	max_depth: 8
wandb: 	min_samples_split: 5
wandb: 	n_estimators: 1024
wandb: 	subsample: 0.5279522934661188
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Agent Starting Run: ut14wyfd with config:
wandb: 	learning_rate: 0.12018630867620815
wandb: 	max_depth: 5
wandb: 	min_samples_split: 8
wandb: 	n_estimators: 1024
wandb: 	subsample: 0.5485592573706465
wandb: wandb version 0.12.0 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade
wandb: Agent Starting Run: jk7optah with config:
wandb: 	learnin

r2,0.95018
_runtime,24
_timestamp,1629194338
_step,3
neg_mape,-0.12918
neg_mae,-677.31669
neg_rmse,-1344.05372


r2,▁
_runtime,▁▁▁▁
_timestamp,▁▁▁▁
_step,▁▃▆█
neg_mape,▁
neg_mae,▁
neg_rmse,▁


r2,0.9496
_runtime,67
_timestamp,1629194409
_step,3
neg_mape,-0.12262
neg_mae,-661.21669
neg_rmse,-1340.06756


...

r2,▁
_runtime,▁▁▁▁
_timestamp,▁▁▁▁
_step,▁▃▆█
neg_mape,▁
neg_mae,▁
neg_rmse,▁


r2,0.94005
_runtime,39
_timestamp,1629198592
_step,3
neg_mape,-0.13884
neg_mae,-740.29768
neg_rmse,-1470.55876


r2,▁
_runtime,▁▁▁▁
_timestamp,▁▁▁▁
_step,▁▃▆█
neg_mape,▁
neg_mae,▁
neg_rmse,▁


r2,0.9531
_runtime,97
_timestamp,1629198693
_step,3
neg_mape,-0.12401
neg_mae,-643.72742
neg_rmse,-1294.51587


r2,▁
_runtime,▁▁▁▁
_timestamp,▁▁▁▁
_step,▁▃▆█
neg_mape,▁
neg_mae,▁
neg_rmse,▁
